# Simple Walk Forward

A simple walk forward implementation based on fixed periods and a single factor

In [11]:
import pandas as pd
import numpy as np
from typing import List, Callable

In [16]:
# Parameters
filename:str ='/tmp/index.csv'
parameters:List[str] = ['trend', 'is_pret']
frequency:str = 'Q'
column:str = 'ret'
function:Callable = np.sum
num:int = 2
sort:bool = False

In [17]:
index = pd.read_csv(filename, parse_dates=['date']).set_index('date')
grouped = index.resample(frequency)

In [18]:
%%time
collect = {}
param_dict = {}
params = {}
for i,(name, group) in enumerate(grouped):  
    if i > 0:
        res = group.copy()
        if params:
            if type(params) == dict:
                for k,v in params.items():
                    res = res[res[k].isin(v)]
            elif type(params) == list:
                query = '|'.join([f"({x})" for x in params])
                res = res.query(query)
            collect[name] = res
            param_dict[name] = params
    s = group.groupby(parameters).agg({column: function})
    select = s.sort_values(by=column, ascending=sort).head(num).index
    if len(select.names) == 1:
        params = {select.name:list(select.values)}
    else:
        params = []
        for values in select.values:
            v = [(x,y) for x,y in zip(select.names, values)]
            v = '&'.join([f"({x}=={y})" for x,y in v])
            params.append(v)
        
result = pd.concat(collect.values())

CPU times: user 1.83 s, sys: 248 µs, total: 1.83 s
Wall time: 1.84 s
